In [ ]:
#!pip install GPUtil
#!pip list | grep -E 'scikit-learn|xgboost'
#!pip install --upgrade scikit-learn

In [2]:
!pip install xgboost
!pip install --upgrade tensorflow
!pip install pytz
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import importlib
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import product
from datetime import datetime
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import forgeall_training_stats_to_csv_appender
importlib.reload(forgeall_training_stats_to_csv_appender)
from forgeall_training_stats_to_csv_appender import *

import forgeall_timestamp_generator
importlib.reload(forgeall_timestamp_generator)
from forgeall_timestamp_generator import *

import forgeall_data_prep
importlib.reload(forgeall_data_prep)
from forgeall_data_prep import *

import forgeall_ssae_model_manager
importlib.reload(forgeall_ssae_model_manager)
from forgeall_ssae_model_manager import *

import forgeall_ssae_model_inferencers
importlib.reload(forgeall_ssae_model_inferencers)
from forgeall_ssae_model_inferencers import *

import forgeall_ssae_model_cross_validator
importlib.reload(forgeall_ssae_model_cross_validator)
from forgeall_ssae_model_cross_validator import *

import forgeall_ssae_model_explainers
importlib.reload(forgeall_ssae_model_explainers)
from forgeall_ssae_model_explainers import *

# Data prep using feature engineered data
Methods in: forgeall_data_prep.py

In [5]:
file_path = 'OpenCalphad_w_melting_point - A.csv'
fa_print_current_timestamp_in_pst()
# Set random seed for reproducibility
np.random.seed(10)

(train_data,
 test_data,
 train_labels,
 test_labels) = fa_prep_hea_data(file_path)

fa_print_current_timestamp_in_pst()

Current Timestamp in PST: 2025-03-05 13:13:43 PST
YS(Mpa)                  1.000000
R(%)                     0.365258
Hom_Temp(K)              0.171878
phase_fraction_fcc       0.159452
Ni(at%)                  0.139874
Co(at%)                  0.117739
CR(%)                    0.116685
Cr(at%)                  0.093100
Mo(at%)                  0.082522
(0-T)*Entropy            0.071566
Cu(at%)                  0.039072
V(at%)                   0.037021
melting point(C)         0.009772
C(at%)                   0.003877
Al(at%)                  0.000190
second_phase_fraction   -0.002055
Anneal_Time(h)          -0.013311
H_RT_FCC                -0.048790
grain(um)               -0.054789
G_RT_FCC                -0.067198
Enthalpy_FCC            -0.067310
Config_Entropy          -0.071566
Mn(at%)                 -0.109061
Fe(at%)                 -0.156773
phase_fraction_bcc      -0.161591
Anneal_Temp(K)          -0.323438
Name: YS(Mpa), dtype: float64
NaN counts per column:
C(wt)        

#TRAINING: the SSAE model.
Methods in: forgeall_ssae_model_manager.py

In [ ]:
fa_print_current_timestamp_in_pst()

best_result = fa_train_ssae_model(train_data,
                                  test_data,
                                  train_labels,
                                  test_labels,
                                  loss_weight_divergence = 0.6,
                                  loss_weight_sparse = 0.4)

fa_print_current_timestamp_in_pst()


###Use the best hyperparam's found from the grid searched model training, fit the data and save the model as a disk file
Methods in: forgeall_ssae_model_manager.py

In [ ]:
# Use the strategy scope
fa_print_current_timestamp_in_pst()
# Create a timestamp in MMDDYYYY_HH_MM_SS format
timestamp = datetime.now().strftime("%m%d%Y_%H_%M_%S")
model_filename_to_save = f"{timestamp}_ssae_model.keras"


fa_create_fit_and_save_model_to_file( model_filename_to_save,
                                      best_result,
                                      train_data,
                                      test_data,
                                      train_labels,
                                      test_labels,
                                      input_dim=25,
                                      hidden_dim=int(best_result['hidden_dim']),
                                      latent_dim=int(best_result['latent_dim']),
                                      target_sparsity=best_result['target_sparsity']
                                    )

fa_print_current_timestamp_in_pst()



#ONCE the MODEL is trained and built, it is saved to a file. NO NEED to retrain. Simply load from file
####Load the model from the file, print the models summary and derive the latent representations: latent_train and latent_test.
Methods in: forgeall_ssae_model_manager.py




In [14]:
model_file = './sparse_autoencoder_model.keras'   # <------- CHANGE this filename to the suitable model file name

fa_print_current_timestamp_in_pst()

loaded_model, model_latent_layer_encoding = fa_load_model_from_file(model_file)
latent_train, latent_train_scaled         = fa_get_latent_prediction_vectors_from_model(model_latent_layer_encoding, train_data)
latent_test, latent_test_scaled           = fa_get_latent_prediction_vectors_from_model(model_latent_layer_encoding, test_data)

fa_print_current_timestamp_in_pst()



Current Timestamp in PST: 2025-03-05 13:26:18 PST


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 25)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 16)             │            416 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ latent_layer (Dense)      │ (None, 6)              │            102 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 16)             │            112 │ latent_layer[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 25)             │            425 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 1)              │              7 │ latent_layer[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,062 (4.15 KB)

 Trainable params: 1,062 (4.15 KB)

 Non-trainable params: 0 (0.00 B)

117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Current Timestamp in PST: 2025-03-05 13:26:19 PST


# INFERENCING: MAE's and R2's for the model.
Methods in: forgeall_ssae_model_inferencers.py

### K Nearest Neighbor (KNN) regressor with Grid search & feature importances

In [15]:
fa_print_current_timestamp_in_pst()
(best_knn,
 knn_predictions_test) = fa_perform_k_nearest_neighbor_regression(latent_train_scaled,
                                                                  latent_test_scaled,
                                                                  train_labels,
                                                                  test_labels)
fa_print_current_timestamp_in_pst()

#----------- Save the KNN model to a file for later use -------------------
timestamp = datetime.now().strftime("%m%d%Y_%H_%M_%S")
scikit_regressor_model_filename_to_save = f"{timestamp}_best_knn_model.pkl"

fa_save_scikit_regressed_model_to_file(best_knn, scikit_regressor_model_filename_to_save)


Current Timestamp in PST: 2025-03-05 13:26:22 PST
Best KNN Hyperparameters: {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 9, 'p': 3, 'weights': 'distance'}

Best KNN Regression Performance on Latent Test Data:
MSE: 0.0026936154881580313
RMSE: 0.05190005287240112
MAE: 0.024834474245241923
R²: 0.9490617361920328
Current Timestamp in PST: 2025-03-05 13:26:40 PST


### XGBoost Regressor

In [ ]:
fa_print_current_timestamp_in_pst()
fa_perform_xgboost_regression (latent_train_scaled, latent_test_scaled, train_labels, test_labels)
fa_print_current_timestamp_in_pst()

### SVM Regressor

In [ ]:
fa_print_current_timestamp_in_pst()
fa_perform_svm_regression(latent_train_scaled, latent_test_scaled, train_labels, test_labels)
fa_print_current_timestamp_in_pst()


### Vanilla KERAS Neural Net Regressor

In [ ]:
fa_print_current_timestamp_in_pst()
fa_perform_kerasnn_regression(latent_train_scaled, latent_test_scaled, train_labels, test_labels)
fa_print_current_timestamp_in_pst()


# CROSS VALIDATION: Tests for over-fitting
Methods in: forgeall_ssae_model_cross_validator.py

In [ ]:
fa_print_current_timestamp_in_pst()
fa_perform_ssae_model_cross_validations(best_knn, latent_train_scaled, train_labels)
fa_print_current_timestamp_in_pst()

# MODEL EXPLAINABILITY
Methods in: forgeall_ssae_model_explainers.py

## Using feature correlations

In [ ]:
fa_print_current_timestamp_in_pst()
fa_explain_ssae_model_using_feature_correlations(best_knn, encoder_model, train_data, test_data, latent_train_scaled, latent_test_scaled, test_labels)
fa_print_current_timestamp_in_pst()

## Another way to perform correlation analysis between original features and important latent features using SHAP

In [ ]:
fa_print_current_timestamp_in_pst()
fa_explain_ssae_model_using_shap(best_knn, encoder_model, train_data, test_data)
fa_print_current_timestamp_in_pst()
